In [1]:
%%capture
#@title ## Prepare environment
#@markdown Execute the cells to load and set up the environment.
#@markdown </br> You can do this by selecting Runtime -> Run all or by pressing CTRL+F9

# Install necessary packages

!pip install -U spacy==2.3.7
!pip3 install pickle5
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

!pip install pyresparser
# Import
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HTML, Button, HBox, VBox

import os
import numpy as np
import sys
import pathlib
import base64
import pandas as pd


from spacy import displacy
import warnings
warnings.filterwarnings('ignore')


# Define necessary paths
src = "nlp/"
storage = "temp_storage/"
ner_path = src + "cnt/trained_model/ner/"

# Download files
if not os.path.exists(src):
  !wget -O file.zip 'https://docs.google.com/uc?export=download&id=1dYLnIKts07SXxsyBo8xx_R1TgErYLj_w&confirm=t'
  !unzip file.zip


# Create a storage folder
if not os.path.exists(storage):
  os.mkdir(storage)

# Remove unecessary files
if os.path.exists(ner_path+".ipynb_checkpoints"):
  os.rmdir(ner_path+".ipynb_checkpoints")

# Imports from downloaded files
sys.path.append(src)
from cnt.model import load_ner_model_v2
from cnt.model import load_pipeline, predict_re_single_sentence

In [2]:
#@title Named Entity Recognition
#@markdown You can select the language and use the trained model to predict the entered design.
#@markdown </br> It is also possible to download the detected entities.
#@markdown </br>
#@markdown </br> [Provide feedback](https://forms.gle/EZSf8gvL8rjWpS6L6)


# data
example_list = ["Head of Athena, right, wearing Corinthian helmet.",
                "Laureate bust of Septimius Severus, right, wearing scale cuirass and paludamentum; gorgoneion on cuirass.",
                "Silenus kneeling left, holding cantharus in his right hand and resting his left on his hip."]
# Editable fields
textfield = widgets.Text(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)


# Buttons
predict_button = widgets.Button(description="Predict")
buttonds = widgets.Button(description="Delete Session")
# Output fields
output = widgets.Output()

# Non editable fields
text_textfield = widgets.HTML(
    value="<b>Design: </b>"
)

text_language = widgets.HTML(
    value="<b>Select language: </b>"
)

text_example = widgets.HTML(
    value="<b>Examples: </b>"
)
# Dropdowns
drop = widgets.Dropdown(
    options=os.listdir(ner_path),
    disabled=False, continuous_update=True,layout={'width': 'max-content'},
    value="english"
)

example_dropdown = widgets.Dropdown(
    options=example_list,
    disabled=False, continuous_update=True,layout={'width': 'max-content'},
    value=None
)

def load_language(language):

  model_directory =  ner_path + drop.value+"/"

  if language == "english":
    model_name = "english_cno"

  elif language== "german":
    model_name = "german_cno"

  return load_ner_model_v2(model_directory, model_name)


# Predict function
def apply_nlp(b):



  MODEL = load_language(drop.value)

  sentence = textfield.value
  if sentence == "":
    sentence = example_dropdown.value

  prediction = MODEL.predict_single_sentence_clear(sentence, as_doc=True)

  colors = {'PERSON': 'mediumpurple','OBJECT': 'greenyellow', 'ANIMAL' : 'orange', 'PLANT': 'salmom', 'VERBS': 'skyblue'}
  options = {'ent': ['PERSON', 'OBJECT', 'ANIMAL', 'PLANT'], 'colors': colors}

  with output:
    output.clear_output()
    print()
    display(sentence)
    displacy.render(prediction,style='ent', jupyter=True, options=options)

  create_csv(sentence)

# create downloadable csv
def create_csv(sentence):
  MODEL = load_language(drop.value)

  prediction1 = MODEL.predict_single_sentence(sentence)
  prediction2 = MODEL.predict_single_sentence_clear(sentence, as_doc=False)

  pred_dict = {}
  for pred in range(len(prediction1)):
    pred_dict[pred] = [prediction2[pred][0], prediction2[pred][1], prediction1[pred][0], prediction1[pred][1]]


  df = pd.DataFrame.from_dict(pred_dict, orient="index", columns=["Entity", "Class", "Start", "End"]).to_csv(index=False)

  filename = 'prediction.csv'
  b64 = base64.b64encode(df.encode())
  payload = b64.decode()

  html = '<a download="{filename}" href="data:text/csv;base64,{payload}"><button class="lm-Widget p-Widget jupyter-widgets jupyter-button widget-button">Download Prediction</button></a>'
  html = html.format(payload=payload, filename=filename)
  with output:
    display(HTML(html))

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        textfield.value=example_dropdown.value

example_dropdown.observe(on_change)


# on_click functions
predict_button.on_click(apply_nlp)

# Display section
display(HBox([text_textfield, textfield, HBox([text_example, example_dropdown])]))
display(HBox([text_language, drop]))
display(predict_button)
display(output)

text_example.add_class("left-spacing-class")
display(HTML(
     "<style>.left-spacing-class {margin-left: 150px;}</style>"
))


Button(description='Predict', style=ButtonStyle())

Output()

HTML(value='<style>.left-spacing-class {margin-left: 150px;}</style>')